In [1]:
# from huggingface_hub import hf_hub_download 
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import torch
from datasets import load_dataset, load_from_disk
import evaluate
import pandas as pd
import os

In [2]:
import transformers
transformers.__version__

'4.30.2'

In [3]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [4]:
cache_dir = '/cygdrive/d/projects/LLM/.cache'
os.environ['TRANSFORMERS_CACHE'] = cache_dir

In [5]:
dataset = load_dataset("yelp_review_full")

Found cached dataset parquet (C:/cygwin64/home/jacky/.cache/huggingface/datasets/parquet/yelp_review_full-e22176106d6e7534/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased", cache_dir=cache_dir)
tokenized_datasets = dataset.map(tokenize_function, batched=True)
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

Loading cached processed dataset at C:\cygwin64\home\jacky\.cache\huggingface\datasets\parquet\yelp_review_full-e22176106d6e7534\0.0.0\14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7\cache-e45664f4cbeacc82.arrow
Loading cached processed dataset at C:\cygwin64\home\jacky\.cache\huggingface\datasets\parquet\yelp_review_full-e22176106d6e7534\0.0.0\14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7\cache-d781ba5f2a8235ba.arrow
Loading cached shuffled indices for dataset at C:\cygwin64\home\jacky\.cache\huggingface\datasets\parquet\yelp_review_full-e22176106d6e7534\0.0.0\14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7\cache-a43f87303ff6e14a.arrow
Loading cached shuffled indices for dataset at C:\cygwin64\home\jacky\.cache\huggingface\datasets\parquet\yelp_review_full-e22176106d6e7534\0.0.0\14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7\cache-de0372290781a33a.arrow


In [7]:
tokenized_datasets.save_to_disk('./dataset')

Saving the dataset (0/5 shards):   0%|          | 0/650000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
# tokenized_datasets = load_from_disk('./dataset')
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

Loading cached shuffled indices for dataset at C:\cygwin64\home\jacky\.cache\huggingface\datasets\parquet\yelp_review_full-e22176106d6e7534\0.0.0\14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7\cache-a43f87303ff6e14a.arrow
Loading cached shuffled indices for dataset at C:\cygwin64\home\jacky\.cache\huggingface\datasets\parquet\yelp_review_full-e22176106d6e7534\0.0.0\14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7\cache-de0372290781a33a.arrow


In [9]:
len(small_train_dataset['input_ids'][0])

512

In [10]:
pd.DataFrame(small_train_dataset['label']).value_counts()

0    209
3    204
1    199
4    197
2    191
dtype: int64

In [11]:
# load the model
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

D:\projects\LLM\env\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\cygwin64\home\jacky\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at google-bert/bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictio

In [12]:
model.to("cuda")

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [14]:
metric = evaluate.load("accuracy")
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [55]:
params = model._modules['bert'].encoder.layer[:-2].parameters()
for param in params:
    param.requires_grad = False

In [70]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [71]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 